## QUBO Variable Flattening

Each binary decision variable \( x_i^k \in \{0, 1\} \) indicates whether vehicle \( i \) selects route \( k \).

To express the QUBO in matrix form, we flatten the 2D variable space into a 1D array:

\[
q = i \cdot t + k
\]

Thus, for \( n \) vehicles and \( t \) routes, the QUBO has \( N = n \cdot t \) variables, indexed \( q \in [0, N-1] \).  
This enables us to construct a standard QUBO matrix \( Q \in \mathbb{R}^{N \times N} \).

---

## Objective Function: Congestion Minimization

Let \( w[i][j][k_1][k_2] \in [0,1] \) be the normalized congestion weight between vehicle \( i \) on route \( k_1 \) and vehicle \( j \) on route \( k_2 \).

The total objective to minimize is:

\[
f(x) = \sum_{i=0}^{n-1} \sum_{j=0}^{n-1} \sum_{k_1=0}^{t-1} \sum_{k_2=0}^{t-1} w[i][j][k_1][k_2] \cdot x_i^{k_1} \cdot x_j^{k_2}
\]

Which is directly encoded into the QUBO matrix as:

\[
Q[q_1, q_2] += w[i][j][k_1][k_2] \quad \text{with } q_1 = i \cdot t + k_1, \quad q_2 = j \cdot t + k_2
\]

---

## Penalty Term: One-Hot Constraint

Each vehicle must select exactly one valid route.  
This is enforced via a quadratic penalty:

\[
P_i(x) = \lambda \left( \sum_{k=0}^{t-1} x_i^k - 1 \right)^2
\]

Expanding:

\[
P_i(x) = \lambda \left( \sum_k x_i^k x_i^k + 2 \sum_{k_1 < k_2} x_i^{k_1} x_i^{k_2} - 2 \sum_k x_i^k + 1 \right)
\]

This leads to the following QUBO terms for each **real route variable** \( q \in \text{valid}(i) \):

- **Diagonal:** \( Q[(q, q)] += \lambda \)
- **Off-diagonal:** \( Q[(q_1, q_2)] += 2\lambda \quad \text{for all valid } q_1 < q_2 \)
- **Linear term:** \( Q[(q, 0)] += -2\lambda \)

The constant term \( +\lambda \) is ignored in QUBO, as it doesn’t affect optimization.

---

## Handling Invalid Routes

Some vehicle–route pairs \( (i, k) \) are invalid.  
These are penalized directly by applying a strong diagonal term to discourage their selection:

\[
Q[(q, q)] += 2 \cdot \lambda \quad \text{if } (i, k) \notin \text{valid\_pairs}
\]

This ensures such routes are not chosen during optimization unless absolutely necessary.

---

## Dynamic Penalty Scaling

Instead of picking a fixed λ (e.g., λ = 10), we **analyze the actual congestion values** in the QUBO and adapt λ accordingly.

For each vehicle and valid route \( q \), we check how "important" that variable is in the objective function by calculating its **total contribution to congestion** (how strongly it interacts with others):

\[
\text{Importance}(q) = \sum_{j} Q[q, j] + Q[j, q]
\]

(Effectively summing the row and column of the QUBO matrix for that variable.)

Then, for all real (valid) route variables:

\[
\lambda = \max_{q \in \text{real routes}} \left( \sum_{j} Q[q, j] + Q[j, q] \right)
\]

This gives you a λ that is **greater than any one variable’s contribution to congestion**, ensuring that violating the constraint (e.g., choosing two routes) is always worse than any potential gain in the objective.

---

## Why This Works

- Automatically **adjusts for the scale** of congestion weights.
- Ensures that **even the worst-case constraint violation** (selecting two routes) incurs a penalty greater than any benefit from objective reduction.


## 🎯 Bias Minimization in One-Hot Constraint (QUBO Form)

### Example: One-Hot Constraint Expansion

The one-hot constraint for vehicle \( i \) choosing exactly one route out of \( t \) is:

\[
\lambda \left( \sum_{k=0}^{t-1} x_i^k - 1 \right)^2
\]

Expanding the square:

\[
\lambda \left( \sum_k x_i^k x_i^k + 2 \sum_{k_1 < k_2} x_i^{k_1} x_i^{k_2} - 2 \sum_k x_i^k + 1 \right)
\]

This gives us:

- Quadratic terms: \( x_i^k x_i^{k'} \)
- Linear terms: \( -2 x_i^k \)
- Diagonal terms: \( x_i^k x_i^k = x_i^k \)
- Constant \( \lambda \) — ignored in QUBO

---

### ✅ Bias Minimization Trick

You can **combine** the diagonal and linear terms for each variable:

\[
Q[(q, q)] += \lambda - 2\lambda = -\lambda
\]

Thus, **you don’t need any `Q[(q, 0)]` linear entries**, and the QUBO becomes simpler and more compact.

---

### ⚠️ But: Not Recommended for D-Wave

While this trick works mathematically, **D-Wave’s solvers (especially hybrid solvers) expect linear and quadratic terms to be separated**, for the following reasons:

- 🔍 **Clarity**: Easier to inspect/debug constraint penalties and objectives
- ⚖️ **Numerical stability**: Solvers handle bias scaling separately from quadratic couplings
- ⚙️ **Better performance**: Especially when using `BinaryQuadraticModel` (BQM) format
- ✅ **Best practice**: Use `linear` and `quadratic` dictionaries with `dimod` or `dwave-system`

---

### ✅ Best Practice for D-Wave

Instead of collapsing everything into the diagonal, use:

```python
from dimod import BinaryQuadraticModel

linear = {q: -2 * lambda_penalty for q in real_route_indices}
quadratic = {(q1, q2): 2 * lambda_penalty for q1, q2 in route_pairs}
bqm = BinaryQuadraticModel(linear, quadratic, offset=lambda_penalty, vartype='BINARY')


### 🔧 Adaptive Penalty Scaling Based on QUBO Size

To enforce the one-hot route assignment constraint without over-penalizing, we scale the penalty parameter λ gently with the size of the QUBO.

Let:
- \( N = n \cdot t \) = total number of binary decision variables
- `max_importance` = maximum row + column sum in the QUBO for real route variables (i.e., strongest interaction)

We define:
\[
\lambda = \left( \frac{N}{10{,}000} \right) \cdot \text{max\_importance}
\]

This gives a moderate penalty increase for larger QUBOs while maintaining balance between constraint satisfaction and objective optimization.

#### ✅ Code Example:

```python
N = n_filtered * t
gamma = N / 10000  # e.g. 1.6 if N = 16,000
lambda_penalty = gamma * max_importance
